In [37]:
## Imports

from PyPDF2 import PdfReader, PdfWriter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
import streamlit as st
import os
from dotenv import load_dotenv

load_dotenv()

genai.configure(api_key=os.getenv("GENAI_API_KEY"))

In [38]:
## Helper Functions 

def get_pdf_text(pdf_path):
    pdf = PdfReader(pdf_path)
    text = ''
    for page in pdf.pages:
        text += page.extract_text()
    return text
def get_vector_store(text):
    embeddings = GoogleGenerativeAIEmbeddings(model='gpt2')
    vector_store = FAISS.from_texts(text,embeddings)
    vector_store.save_local('faiss_index')
def get_conversational_chain():
    prompt_template = 'Q: {question}\nA: {answer}\n'
    model = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.5)
    prompt = PromptTemplate(template= prompt_template,input_variables=["context","question"])
    chain = load_qa_chain(model,chain_type = "stuff",prompt = prompt)
    return chain
def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    new_db = FAISS.load_local("faiss_index", embeddings)
    docs = new_db.similarity_search(user_question)
    chain = get_conversational_chain()
    
    response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)

    print(response)


In [44]:
import streamlit as st

def main():
    st.set_page_config(page_title="Chat PDF", page_icon=":file_pdf:")  # Set title and icon

    # Add custom CSS
    st.markdown("""
        <style>
        .custom-header {
            color: #3498db;
            text-align: center;
        }
        .custom-sidebar-title {
            color: #2ecc71;
        }
        .custom-text-input {
            font-size: 18px;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 5px;
        }
        .custom-button {
            background-color: #9b59b6;
            color: white;
            padding: 10px 20px;
            border: none;
            border-radius: 5px;
        }
        </style>
    """, unsafe_allow_html=True)

    # Colored header with center alignment
    st.markdown("<h1 class='custom-header'>Chat with PDF using Gemini</h1>", unsafe_allow_html=True)

    # Styled text input
    user_question = st.text_input("**Ask a Question from the PDF Files**", key="question_input")

    if user_question:
        user_input(user_question)

    with st.sidebar:
        # Green colored sidebar title
        st.markdown("<h3 class='custom-sidebar-title'>Menu:</h3>", unsafe_allow_html=True)

        pdf_docs = st.file_uploader("Upload your PDF Files and Click on the Submit & Process Button", accept_multiple_files=True)

        # Styled button
        if st.button("Submit & Process"):
            with st.spinner("Processing..."):
                raw_text = get_pdf_text(pdf_docs)
                get_vector_store(raw_text)
            st.success("Done!")

if __name__ == "__main__":
    main()

2024-09-28 22:38:19.114 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 22:38:19.114 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 22:38:19.115 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 22:38:19.116 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 22:38:19.116 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 22:38:19.117 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 22:38:19.117 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 22:38:19.118 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar